# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Huỳnh Tấn Vinh
- MSSV: 20127666
- Lớp: 20CLC08

# Import

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt


# Import thêm dữ thư viện nếu cần

In [2]:
import pandas as pd
train = pd.read_csv('train.csv')
train.head()

,Adult Mortality,BMI,Polio,Diphtheria,HIV/AIDS,GDP,Thinness age 10-19,Thinness age 5-9,Income composition of resources,Schooling,Life expectancy
0,268.0,18.1,62.0,64.0,0.1,631.744976,17.7,17.7,0.470,9.9,59.9
1,272.0,17.6,67.0,67.0,0.1,669.959000,17.9,18.0,0.463,9.8,59.5
2,275.0,17.2,68.0,68.0,0.1,63.537231,18.2,18.2,0.454,9.5,59.2
3,279.0,16.7,66.0,66.0,0.1,553.328940,18.4,18.4,0.448,9.2,58.8
4,281.0,16.2,63.0,63.0,0.1,445.893298,18.6,18.7,0.434,8.9,58.6


# Đọc dữ liệu

In [3]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [4]:
# Cài đặt các hàm cần thiết ở đây
xtrain = np.array(X_train)
ytrain = np.array(y_train)

xtest = np.array(X_test)
ytest = np.array(y_test)



In [5]:
class OLSLinearRegression:
    # 
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T
        self.w = X_pinv @ y
        return self
    
    def get_params(self):
        return self.w
    
    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)  

In [6]:
#Tính lỗi trung bình phương gốc (root-mean-square - RMSE)
def RMSE(y, y_hat):
    return np.sqrt(np.mean((y.ravel() - y_hat.ravel())**2))

# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [7]:
# Phần code cho yêu cầu 1a
lr = OLSLinearRegression().fit(xtrain, ytrain)
lr.get_params()

array([ 1.51013627e-02,  9.02199807e-02,  4.29218175e-02,  1.39289117e-01,
       -5.67332827e-01, -1.00765115e-04,  7.40713438e-01,  1.90935798e-01,
        2.45059736e+01,  2.39351661e+00])

In [8]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
y_hat = lr.predict(xtest) #kết quả của chương trình dự đoán 
RMSE(ytest, y_hat) #y_test 

7.064046430584466

Công thức hồi quy

$$\text{Life expectancy} = (1.51013627e-02 * Adult Mortality) + (9.02199807e-02 * BMI) + (4.29218175e-02 * Polio) + (1.39289117e-01 * Diphtheria) + (-5.67332827e-01 * HIV/AIDS) + (-1.00765115e-04 * GDP) + (7.40713438e-01 * Thinness age 10-19) + (1.90935798e-01 * Thinness age 5-9) + (2.45059736e+01 * Income composition of resources) + (2.39351661e+00 * Schooling)$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [9]:
# Phần code cho yêu cầu 1b
# In ra các kết quả cross-validation như yêu cầu
def CrossValidationHandle(X_train_feature, y_train_feature, split_fold):  
    
    number_of_sample = X_train_feature.shape[0]
    set_n_data = number_of_sample / split_fold
    rmse_temp = 0
    rmse = 0
    for i in range (split_fold):
        _start = int(i * set_n_data)
        _end = int((i + 1) * set_n_data)
        
        X_value = X_train_feature[_start:_end]
        y_value = y_train_feature[_start:_end]
        
        X_Ktrain_feature = np.array([*X_train_feature[:_start], *X_train_feature[_end:]]) 
        y_Ktrain_feature = np.array([*y_train_feature[:_start], *y_train_feature[_end:]])
        lr.fit(X_Ktrain_feature, y_Ktrain_feature)
        
        y_value_predict = lr.predict(X_value)
        
        rmse_temp += RMSE(y_value, y_value_predict)
        rmse = rmse_temp / split_fold 
    return rmse
    

In [10]:
# Tìm ra đặc trưng tốt nhất
number_of_lines = X_train.shape[0]
array_line = list(range(0, number_of_lines))
random.shuffle(array_line)
rmse_value = np.zeros(10)
split_fold = 5

X_train_clone = []
y_train_clone = []

for index in range(X_train.shape[0]):
    X_train_clone.append(xtrain[array_line[index]])
    y_train_clone.append(ytrain[array_line[index]])

X_train_clone = np.array(X_train_clone)
y_train_clone = np.array(y_train_clone) 

for index in range(10):
    X_train_feature = X_train_clone[:, index].copy().reshape((-1, 1))
    y_train_feature = y_train_clone.copy()
    rmse_value[index] = CrossValidationHandle(X_train_feature, y_train_feature, split_fold)
index_best_feature = np.where(rmse_value == np.amin(rmse_value))[0][0]


In [11]:
# In ra các kết quả cross-validation như yêu cầu
print("RMSE: ", rmse_value)

#In ra đặc trưng tốt nhất
print("Vị trí của đặc trưng tốt nhất: ", index_best_feature) 



RMSE:  [46.24043914 27.93630868 18.01605785 16.02689974 67.06364447 60.36394626
 51.83799156 51.76711466 13.31283534 11.786606  ]
Vị trí của đặc trưng tốt nhất:  9


In [12]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
X_train_best_feature = X_train_clone[:,index_best_feature].copy().reshape((-1,1))
lr.fit(X_train_best_feature, y_train_clone)
lr.get_params()


array([5.5573994])

In [13]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
y_best_predict = lr.predict(X_test.to_numpy()[:, index_best_feature].reshape(-1, 1))
RMSE(y_test.copy(), y_best_predict)

10.260950391655376

Công thức hồi quy

$$\text{Life expectancy} = Schooling * 5.5573994$$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

In [14]:
#Tính độ tương quan
corr = 0
for i in range (10):
    corr = np.corrcoef(xtrain[:,i], ytrain)[0, 1]
    print(corr)

-0.685290066876047
0.6106869049325891
0.36570463313391016
0.40679787281341867
-0.5865777200247837
0.4867608561696108
-0.5036523346464049
-0.5002309457544405
0.779403572108283
0.7548637211053559


In [15]:
corr = 0
for i in range (10):
    corr = np.corrcoef(xtrain[:,i]**3, ytrain)[0, 1]
    print(corr)

-0.589889204315234
0.6774872626372263
0.4929196499694736
0.5234607846884396
-0.3080360984786394
0.3108651003258309
-0.17045510027002106
-0.1646907130960573
0.8300236793420632
0.708830248016597


Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

In [16]:
# Phần code cho yêu cầu 1c
# Paradigm: Income composition of resources, Schooling
rmse1 = 0
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
X_train_clone1 = []
y_train_clone1 = []

number_of_lines = X_train.shape[0]
array_line = list(range(0, number_of_lines))
random.shuffle(array_line)

for index in range(X_train.shape[0]):
    X_train_clone1.append(xtrain[array_line[index]])
    y_train_clone1.append(ytrain[array_line[index]])

X_train_clone1 = np.array(X_train_clone)
y_train_clone1 = np.array(y_train_clone)

X_train_feature_Income = X_train_clone1[:, 8].copy().reshape((-1, 1))
X_train_feature_Schooling = X_train_clone1[:, 9].copy().reshape((-1, 1))

X_train_feature1 = np.column_stack((X_train_feature_Income, X_train_feature_Schooling))
y_train_feature = y_train_clone1.copy()

split_fold = 5

rmse1 = CrossValidationHandle(X_train_feature1, y_train_feature, split_fold)

# In ra các kết quả cross-validation như yêu cầu
print("RMSE 1: ", rmse1)


RMSE 1:  11.237448339156789


In [17]:
# Paradigm: BMI, Income composition of resources, Schooling
rmse2 = 0
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
X_train_feature_BMI = X_train_clone[:, 1].copy().reshape((-1, 1))
X_train_feature_Diphtheria = X_train_clone[:, 3].copy().reshape((-1, 1))
X_train_feature_Income = X_train_clone[:, 8].copy().reshape((-1, 1))
X_train_feature_Schooling = X_train_clone[:, 9].copy().reshape((-1, 1))

X_train_feature2 = np.column_stack((X_train_feature_BMI, X_train_feature_Diphtheria, X_train_feature_Income, X_train_feature_Schooling))
y_train_feature = y_train_clone.copy()

split_fold = 5

rmse2 = CrossValidationHandle(X_train_feature2, y_train_feature, split_fold)

# In ra các kết quả cross-validation như yêu cầu
print("RMSE 2: ", rmse2)

RMSE 2:  9.677123828891265


In [18]:
# Paradigm: Adult Mortality, HIV/AIDS, Thinness age 10-19, Thinness age 5-9
rmse3 = 0
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
X_train_feature_Adult = X_train_clone[:, 0].copy().reshape((-1, 1))
X_train_feature_HIV_AIDS = X_train_clone[:, 4].copy().reshape((-1, 1))
X_train_feature_Thinness_age_10_19 = X_train_clone[:, 6].copy().reshape((-1, 1))
X_train_feature_Thinness_age_5_9 = X_train_clone[:, 7].copy().reshape((-1, 1))

X_train_feature3 = np.column_stack((X_train_feature_Adult,X_train_feature_HIV_AIDS, X_train_feature_Thinness_age_10_19,X_train_feature_Thinness_age_5_9))
y_train_feature = y_train_clone.copy()

split_fold = 5

rmse3 = CrossValidationHandle(X_train_feature3, y_train_feature, split_fold)

# In ra các kết quả cross-validation như yêu cầu
print("RMSE 3: ", rmse3)

RMSE 3:  41.6291079301479


In [19]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
X_train_best_feature2 = X_train_feature2
lr.fit(X_train_best_feature2, y_train_clone)
lr.get_params()

array([-6.40259278e-03,  2.50004203e-01,  2.78218706e+01,  2.44185275e+00])

In [20]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
# y_best_predict = lr.predict()
X_train_feature_BMI = X_test.to_numpy()[:, 1].copy().reshape((-1, 1))
X_train_feature_Diphtheria = X_test.to_numpy()[:, 3].copy().reshape((-1, 1))
X_train_feature_Income = X_test.to_numpy()[:, 8].copy().reshape((-1, 1))
X_train_feature_Schooling = X_test.to_numpy()[:, 9].copy().reshape((-1, 1))
y_best_predict = lr.predict(np.column_stack((X_train_feature_BMI,X_train_feature_Diphtheria,X_train_feature_Income, X_train_feature_Schooling)))
RMSE(y_test.copy(), y_best_predict)

8.280332718815753

Công thức hồi quy

$$\text{Life expectancy} = (BMI * -6.40259278e-03) + (Diphtheria * 2.50004203e-01) + (Income composition of resources * 2.78218706e+01) + (Schooling * 2.44185275e+00)$$